In [1]:

# from model.embeddings import text_embedding_3large
import os
from langchain.vectorstores.pgvector import PGVector
from langchain.indexes import SQLRecordManager, index
from langchain_core.documents.base import Document
import pandas as pd

import psycopg
import os
from dotenv import load_dotenv
load_dotenv("../application/.env")


True

In [2]:
from langchain_openai import AzureChatOpenAI
from langchain_openai import OpenAIEmbeddings, AzureOpenAIEmbeddings

In [3]:
host = os.getenv("PG_VECTOR_HOST")
user = os.getenv("PG_VECTOR_USER")
password = os.getenv("PG_VECTOR_PASSWORD")

# database = os.getenv("PGDATABASE3") # 22 Jul 2024
# database = os.getenv("PGDATABASE4") # 23 Jul 2024
# database = os.getenv("PGDATABASE5") # 12 Aug 2024
database = os.getenv("PGDATABASE2") # 16 Aug 2024
# COLLECTION_NAME = "buv_general_info"

# CONNECTION_STRING = f"postgresql+psycopg2://{user}:{password}@{host}:5432/{database}"
CONNECTION_STRING = f"postgresql+psycopg://{user}:{password}@{host}:5432/{database}"

In [4]:
database

'bus_schedule_16082024'

In [5]:
# Azure model
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

azure_openai = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    azure_deployment="apac-gpt-35-turbo",
    temperature=0
)

text_embedding_3large = AzureOpenAIEmbeddings(
    model="apac-text-embedding-3-large",
    openai_api_version="2022-12-01",
    openai_api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
)

In [6]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, DateTime, ForeignKey, Text
import pandas as pd

In [7]:
# Thông tin kết nối tới PostgreSQL
db_username = user
db_password = password
db_host = host
db_port = 5432
db_name = database

# Tạo kết nối tới PostgreSQL
# engine = create_engine(f'postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')
engine = create_engine(f'postgresql+psycopg://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

In [8]:
# Tạo metadata object
metadata = MetaData()

In [9]:

# Định nghĩa các bảng
bus_trips = Table('bus_trips', metadata,
                  Column('trip_id', Integer, primary_key=True),
                  Column('route', Text, nullable=False),
                  Column('departure_district', Text, nullable=False),
                  Column('arrival', Text, nullable=False),
                  Column('departure_time', DateTime, nullable=False),
                  Column('arrival_time', DateTime, nullable=False))

bus_schedule = Table('bus_schedule', metadata,
                     Column('trip_id', Integer, ForeignKey('bus_trips.trip_id'), primary_key=True),
                     Column('stop_sequence', Integer, primary_key=True),
                     Column('stop_name', Text),
                     Column('stop_time', DateTime))

bus_timetable = Table('bus_timetable', metadata,
                      Column('day_of_week', Text, primary_key=True),
                      Column('trip_id', Integer, ForeignKey('bus_trips.trip_id'), primary_key=True),
                      Column('date', DateTime))


In [10]:
# metadata.drop_all(engine)

# Tạo các bảng trong cơ sở dữ liệu
metadata.create_all(engine)

In [13]:
# Đọc dữ liệu từ các file CSV
# bus_schedule_df = pd.read_csv('../data/csv/bus_schedule_20240816.csv')
# bus_timetable_df = pd.read_csv('../data/csv/bus_timetable_20240816.csv')
# bus_trips_df = pd.read_csv('../data/csv/bus_trips_20240816.csv')

# bus_schedule_df = pd.read_csv('../data/csv/bus_schedule_20240818.csv')
# bus_timetable_df = pd.read_csv('../data/csv/bus_timetable_20240818.csv')
# bus_trips_df = pd.read_csv('../data/csv/bus_trips_20240818.csv')

bus_schedule_df = pd.read_csv('../data/csv/bus_schedule_20240818.csv')
bus_timetable_df = pd.read_csv('../data/csv/bus_timetable_20240818.csv')
bus_trips_df = pd.read_csv('../data/csv/bus_trips_20240818.csv')

In [14]:
bus_timetable_df.shape

(250, 3)

In [15]:
# Đổi tên các cột để khớp với tên cột trong cơ sở dữ liệu
bus_schedule_df.columns = ['trip_id', 'stop_sequence', 'stop_name', 'stop_time']
bus_timetable_df.columns = ['day_of_week', 'trip_id', 'date']
bus_trips_df.columns = ['trip_id', 'route', 'departure_district', 'arrival', 'departure_time', 'arrival_time']

In [16]:
# Đảm bảo các cột có định dạng đúng
bus_schedule_df['stop_time'] = pd.to_datetime(bus_schedule_df['stop_time'])
bus_timetable_df['date'] = pd.to_datetime(bus_timetable_df['date'], format='%d/%m/%Y')
bus_trips_df['departure_time'] = pd.to_datetime(bus_trips_df['departure_time'])
bus_trips_df['arrival_time'] = pd.to_datetime(bus_trips_df['arrival_time'])

/var/folders/cz/t0wnr3c949j4t87gbzk2qkzc0000gn/T/ipykernel_61086/1917986169.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  bus_schedule_df['stop_time'] = pd.to_datetime(bus_schedule_df['stop_time'])
/var/folders/cz/t0wnr3c949j4t87gbzk2qkzc0000gn/T/ipykernel_61086/1917986169.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  bus_trips_df['departure_time'] = pd.to_datetime(bus_trips_df['departure_time'])
/var/folders/cz/t0wnr3c949j4t87gbzk2qkzc0000gn/T/ipykernel_61086/1917986169.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  bus_trips_df['arrival_time'] = pd.to_dat

In [17]:
# Tải dữ liệu lên PostgreSQL
bus_trips_df.to_sql('bus_trips', engine, if_exists='append', index=False)
bus_schedule_df.to_sql('bus_schedule', engine, if_exists='append', index=False)
bus_timetable_df.to_sql('bus_timetable', engine, if_exists='append', index=False)

print("Dữ liệu đã được tải lên PostgreSQL thành công!")

IntegrityError: (psycopg.errors.UniqueViolation) duplicate key value violates unique constraint "bus_trips_pkey"
DETAIL:  Key (trip_id)=(1) already exists.
[SQL: INSERT INTO bus_trips (trip_id, route, departure_district, arrival, departure_time, arrival_time) VALUES (%(trip_id)s::BIGINT, %(route)s::VARCHAR, %(departure_district)s::VARCHAR, %(arrival)s::VARCHAR, %(departure_time)s::TIMESTAMP WITHOUT TIME ZONE, %(arrival_time)s::TIMESTAMP WITHOUT TIME ZONE)]
[parameters: [{'trip_id': 1, 'route': 'Hai Ba Trung to BUV Campus', 'departure_district': 'Hai Ba Trung', 'arrival': 'BUV Campus', 'departure_time': datetime.datetime(2024, 8, 19, 7, 40), 'arrival_time': datetime.datetime(2024, 8, 19, 8, 25)}, {'trip_id': 2, 'route': 'Hai Ba Trung to BUV Campus', 'departure_district': 'Hai Ba Trung', 'arrival': 'BUV Campus', 'departure_time': datetime.datetime(2024, 8, 19, 9, 40), 'arrival_time': datetime.datetime(2024, 8, 19, 10, 25)}, {'trip_id': 3, 'route': 'Hai Ba Trung to BUV Campus', 'departure

In [47]:
# Truy vấn dữ liệu từ bảng bus_trips
query = f"""
select * from bus_timetable;
--select * from bus_schedule;
"""

In [48]:
df_ = pd.read_sql(query, engine)
print("Dữ liệu từ bảng:")
print(df_)

Dữ liệu từ bảng:
    day_of_week  trip_id       date
0        Monday        1 2024-08-12
1        Monday        3 2024-08-12
2        Monday        4 2024-08-12
3        Monday        6 2024-08-12
4        Monday        8 2024-08-12
..          ...      ...        ...
229    Saturday       49 2024-08-17
230    Saturday       50 2024-08-17
231    Saturday       53 2024-08-17
232    Saturday       54 2024-08-17
233    Saturday       55 2024-08-17

[234 rows x 3 columns]


In [49]:
df_.head(55)

,day_of_week,trip_id,date
0,Monday,1,2024-08-12
1,Monday,3,2024-08-12
2,Monday,4,2024-08-12
3,Monday,6,2024-08-12
4,Monday,8,2024-08-12
5,Monday,9,2024-08-12
6,Monday,11,2024-08-12
7,Monday,13,2024-08-12
8,Monday,14,2024-08-12
9,Monday,16,2024-08-12


In [50]:
# Truy vấn dữ liệu từ bảng bus_trips
query = f"""
select * from bus_schedule;
"""

In [51]:
# query = f"""
# select * from bus_schedule;
# """
# query = f"""
#     select date(tmp3.date), src.trip_id,src.stop_name,  time(src.stop_time)  from bus_schedule src
#     join
#     (
#     select a.trip_id, a.stop_sequence from bus_schedule a
#     join bus_timetable b on a.trip_id = b.trip_id
#     where b.date = '2024-05-29 00:00:00'
#     and a.stop_name = 'Nguyen Quoc Tri'
#     and time(a.stop_time) <= '08:00:00'
#     ) tmp1 on src.trip_id = tmp1.trip_id
#     join
#     (
#     select a.trip_id, a.stop_sequence from bus_schedule a
#     join bus_timetable b on a.trip_id = b.trip_id
#     where b.date = '2024-05-29 00:00:00'
#     and a.stop_name = 'BUV Campus'
#     ) tmp2 on src.trip_id = tmp2.trip_id
#     join bus_timetable tmp3 on src.trip_id = tmp3.trip_id
#     where src.stop_sequence >= tmp1.stop_sequence and src.stop_sequence <= tmp2.stop_sequence
#     and tmp3.date = '2024-05-29 00:00:00';
# """


query=f"""
SELECT DATE(tmp3.date) AS date, src.trip_id, src.stop_name, src.stop_time::time AS stop_time
    FROM bus_schedule AS src
    JOIN
    (
    select a.trip_id, a.stop_sequence from bus_schedule a
    join bus_timetable b on a.trip_id = b.trip_id
    where b.date = '2024-05-28 00:00:00'
    and a.stop_name = 'Ho Tay Water Park'
    AND a.stop_time::time = '11:50:00'
    ) tmp1 on src.trip_id = tmp1.trip_id
    join
    (
    select a.trip_id, a.stop_sequence from bus_schedule a
    join bus_timetable b on a.trip_id = b.trip_id
    where b.date = '2024-05-28 00:00:00'
    and a.stop_name = 'BUV Campus'
    ) tmp2 on src.trip_id = tmp2.trip_id
    join bus_timetable tmp3 on src.trip_id = tmp3.trip_id
    where src.stop_sequence >= tmp1.stop_sequence and src.stop_sequence <= tmp2.stop_sequence
    and tmp3.date = '2024-05-28 00:00:00';
"""


# # # bus_trips_df = pd.read_sql('SELECT * FROM bus_trips', engine)
# bus_trips_df = pd.read_sql(query, engine)
# print("Dữ liệu từ bảng:")
# print(bus_trips_df)

In [52]:
bus_trips_df = pd.read_sql(query, engine)
print("Dữ liệu từ bảng:")
print(bus_trips_df)

Dữ liệu từ bảng:
Empty DataFrame
Columns: [date, trip_id, stop_name, stop_time]
Index: []


In [53]:
query = f"""SELECT DATE(tmp3.date) AS date, src.trip_id, src.stop_name, src.stop_time::time AS stop_time
    FROM bus_schedule AS src
    JOIN
    (
    select a.trip_id, a.stop_sequence from bus_schedule a
    join bus_timetable b on a.trip_id = b.trip_id
    where b.date = '2024-07-22 00:00:00'
    and a.stop_name = 'BUV Campus'
    AND a.stop_time::time = '11:40:00'
    ) tmp1 on src.trip_id = tmp1.trip_id
    join
    (
    select a.trip_id, a.stop_sequence from bus_schedule a
    join bus_timetable b on a.trip_id = b.trip_id
    where b.date = '2024-07-22 00:00:00'
    and a.stop_name = 'BUV Campus'
    ) tmp2 on src.trip_id = tmp2.trip_id
    join bus_timetable tmp3 on src.trip_id = tmp3.trip_id
    where src.stop_sequence >= tmp1.stop_sequence and src.stop_sequence <= tmp2.stop_sequence
    and tmp3.date = '2024-07-22 00:00:00';
"""

In [54]:
bus_trips_df = pd.read_sql(query, engine)
print("Dữ liệu từ bảng:")
print(bus_trips_df)

Dữ liệu từ bảng:
Empty DataFrame
Columns: [date, trip_id, stop_name, stop_time]
Index: []


In [55]:
query = f""" SELECT DATE(tmp3.date) AS date, src.trip_id, src.stop_name, src.stop_time::time AS stop_time
    FROM bus_schedule AS src
    JOIN
    (
    select a.trip_id, a.stop_sequence from bus_schedule a
    join bus_timetable b on a.trip_id = b.trip_id
    where b.date = '2024-08-12 00:00:00'
    and a.stop_name = 'Nguyen Quoc Tri'
    AND a.stop_time::time = '07:20:00'
    ) tmp1 on src.trip_id = tmp1.trip_id
    join
    (
    select a.trip_id, a.stop_sequence from bus_schedule a
    join bus_timetable b on a.trip_id = b.trip_id
    where b.date = '2024-08-12 00:00:00'
    and a.stop_name = 'BUV Campus'
    ) tmp2 on src.trip_id = tmp2.trip_id
    join bus_timetable tmp3 on src.trip_id = tmp3.trip_id
    where src.stop_sequence >= tmp1.stop_sequence and src.stop_sequence <= tmp2.stop_sequence
    and tmp3.date = '2024-08-12 00:00:00';

"""

In [56]:
bus_trips_df = pd.read_sql(query, engine)
print("Dữ liệu từ bảng:")
print(bus_trips_df)

Dữ liệu từ bảng:
         date  trip_id               stop_name stop_time
0  2024-08-12        6         Nguyen Quoc Tri  07:20:00
1  2024-08-12        6  National Cinema Centre  07:35:00
2  2024-08-12        6              BUV Campus  08:25:00
